In [1]:
import numpy as np
import matplotlib
matplotlib.use("MACOSX")
import matplotlib.pyplot as plt
import scipy.optimize as optimize


In [2]:
def test_fun(x):
    x.shape=(-1, 1)
    return .5*x[0, 0]**2 + x[0, 0]* np.cos(x[1, 0])

def test_jac(x):
    x.shape=(-1, 1)
    return np.array([[x[0, 0] + np.cos(x[1, 0])],
                      [-x[0, 0] * np.sin(x[1, 0])]])

def test_hess(x):
    x.shape=(-1, 1)
    return np.array([[           1.,        -np.sin(x[1, 0])],
                     [-np.sin(x[1, 0]), -x[0, 0] * np.cos(x[1, 0])]])

In [3]:
fig,ax = plt.subplots()

xg, yg = np.linspace(-2, 2, 51), np.linspace(-6, 6, 51)
def mat_fun(x_g, x_):
    Z = np.zeros((xg.size, yg.size))

    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            Z[j, i] = test_fun(np.array([xg[i], yg[j]]))
    return Z
X, Y = np.meshgrid(xg,yg)
ax.contour(X, Y, mat_fun(xg, yg))
fig.show()

In [4]:
n = 2 # Dimension of x

In [5]:
gamma = 1
m = 3
S = np.zeros((n,0))
Y = np.zeros((n,0))
R = np.zeros((0,0))
STgrad=np.array((0,m))
YTgrad=np.array((0,m))

In [6]:
S.shape

(2, 0)

In [7]:
# Initial history:
x_old = np.array([[0],[1]])
grad_old = test_jac(x_old)

# line search
step =optimize.minimize_scalar(fun = lambda alpha : test_fun(x_old  - grad_old * alpha) ,bounds=(0,2),method="bounded" ).x
x     = x_old - grad_old * step
assert test_fun(x) < test_fun(x_old)
grad = test_jac(x)

k=1

# Start loop

In [41]:
# Update Sk,Yk

if k > m:
    S=np.roll(S,-1)
    S[:,-1] = (x - x_old).flat
    Y=np.roll(Y,-1)
    Y[:,-1]  = (grad - grad_old).flat
    
else :
    S = np.hstack([S,x - x_old])
    Y = np.hstack([Y,grad - grad_old])
print("S: {}".format(S))
print("Y: {}".format(Y))

S: [[-0.69665812  0.1998545   0.03863815]
 [-0.8279051  -0.20718687  0.03331417]]
Y: [[-0.25173224  0.21401065  0.03925223]
 [-0.24282335 -0.24821198  0.03461154]]


In [42]:
(x - x_old).flat

In [43]:
#2.
grad2prev = grad2
grad2 = np.sum(np.asarray(grad)**2) #ok
YTgrad_prev = STgrad.copy() # scalar
YTgrad_prev = YTgrad.copy()
STgrad = np.dot(S.T,grad)
YTgrad = np.dot(Y.T,grad)

In [46]:
print(STgrad)
print(YTgrad)

[[4.03258101e-04]
 [6.73677595e-04]
 [6.08274186e-09]]
[[ 4.57173114e-05]
 [ 7.68168192e-04]
 [-1.35691840e-06]]


In [47]:
#3. # TOOPTIMIZE
sprevTgradprev =  np.dot(S[:,-1].T,grad_old) # sk-1T gk-1


In [49]:
#4. 
ykm12 = np.dot(Y[:,-1].T,Y[:,-1])

print("before")
print("R: {}".format(R))
if k > m:
    R = np.roll(R,(-1,-1),axis=(0,1))# mxm Matrix hold by all Processors 
    R[-1,:] = 0
    R[:,-1] = np.dot(S.T, Y[:,-1])# TOOPTIMIZE
elif k ==1:
    R= np.array(np.dot(S.T,Y))
else :
    R = np.vstack([R, np.zeros(k-1)])
    R = np.hstack([R,np.dot(S.T, Y[:,-1]).reshape(k,1)])
    
if k> m: 
    D = np.roll(D,(-1,-1),axis=(0,1)) 
    #D[-1,-1] = np.dot(Y[:,-1],Y[:,-1])# yk-1Tyk-1 # TOOPTIMIZE
    D[-1,-1] = grad2prev - grad2 + 2 * YTgrad[-1]
    np.testing.assert_almost_equal(D[-1,-1],np.dot(Y[:,-1],Y[:,-1]))
else:
    D = np.diag(np.einsum("ik,ik -> k",S,Y))

#YTY = np.dot(Y.T,Y) #TOOPTIMIZE
if k> m :
    YTY = np.roll(YTY,(-1,-1),axis=(0,1)) 
    print(YTgrad)
    print(YTgrad_prev)
    YTY[-1,:-1] = YTY[:-1,-1] = (YTgrad[:-1] - YTgrad_prev[1:]).flat
    YTY[-1,-1] = D[-1,-1]
else:
    YTY = np.dot(Y.T,Y)
np.testing.assert_allclose(YTY,np.dot(Y.T,Y))

print("after")
print("R: {}".format(R))
print("YTY: {}".format(YTY))
print("D: {}".format(D))

before
R: [[ 0.37640599  0.05640371 -0.05600046]
 [ 0.          0.09419725  0.00067368]
 [ 0.          0.          0.00266969]]
[[ 4.57173114e-05]
 [ 7.68168192e-04]
 [-1.35691840e-06]]
[[0.00383793]
 [0.01833126]
 [0.00095877]]


AttributeError: module 'numpy.testing' has no attribute 'assert_all_close'

In [50]:
YTgrad

array([[ 4.57173114e-05],
       [ 7.68168192e-04],
       [-1.35691840e-06]])

In [51]:
#5.
gamma = D[-1,-1] / ykm12# n.b. D[-1,-1] = sk-1T yk-1 = yk-1T sk-1

In [52]:
#6. 
Rinv = np.linalg.inv(R) # TODO: profitiert das davon dass R eine Dreiecksmatrix ist ?

RiSg = Rinv.dot(STgrad)

p = np.vstack([Rinv.T.dot(D+gamma*YTY).dot(RiSg) - gamma * Rinv.T.dot(YTgrad)
               ,- RiSg])

In [53]:
#7.
Hgrad = gamma*grad + np.hstack([S,gamma*Y]).dot(p) 

In [54]:
# linesearch

alpha =optimize.minimize_scalar(fun = lambda alpha : test_fun(x  - Hgrad * alpha) ,bounds=(0,10),method="bounded" ).x
x_old= x 
x     = x  - Hgrad * alpha
assert test_fun(x) < test_fun(x_old)
grad_old=grad
grad = test_jac(x)

ax.plot(x[0],x[1],"+k")
ax.annotate(k,x)
k = k+1
fig.canvas.draw()

AssertionError: 

In [ ]:
np.testing